In [23]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [30]:
import networkx as nx
from src.utils import load_json
from src.config import filepaths
from src.post_processing import Expansion

In [31]:
imt_reverses = load_json(filepaths['rules'] / "jnimt_reverses.json")
forward = '1_steps_alpha_ketoglutarate_to_None_rules_JN3604IMT_rules_carbonyl_free_co_metacyc_coreactants_carbonyl_free_sampled_False_pruned_False'
reverse = '1_steps_hopa_to_None_rules_JN3604IMT_rules_carbonyl_free_co_metacyc_coreactants_carbonyl_free_sampled_False_pruned_False'

# forward = '2_steps_ccm_v0_to_None_rules_JN3604IMT_rules_co_metacyc_coreactants_sampled_False_pruned_False'
# reverse = '2_steps_bottle_targets_24_to_None_rules_JN3604IMT_rules_co_metacyc_coreactants_sampled_False_pruned_False'

print("Loading expansion")
pk = Expansion(
    forward=filepaths['raw_expansions'] / f"{forward}.pk" if forward else forward,
    reverse=filepaths['raw_expansions'] / f"{reverse}.pk" if reverse else reverse,
    operator_reverses=imt_reverses,
)

Loading expansion


[16:29:57] WARNING: not removing hydrogen atom without neighbors
[16:29:57] WARNING: not removing hydrogen atom without neighbors
[16:29:58] WARNING: not removing hydrogen atom without neighbors
[16:29:58] WARNING: not removing hydrogen atom without neighbors


In [32]:
paths = pk.find_paths()

In [26]:
fwd = Expansion(
    forward=filepaths['raw_expansions'] / f"{forward}.pk"
)

rev = Expansion(
    reverse=filepaths['raw_expansions'] / f"{reverse}.pk",
    operator_reverses=imt_reverses,
)

[16:22:56] WARNING: not removing hydrogen atom without neighbors
[16:22:56] WARNING: not removing hydrogen atom without neighbors
[16:22:58] WARNING: not removing hydrogen atom without neighbors
[16:22:58] WARNING: not removing hydrogen atom without neighbors


In [27]:
len(fwd.compounds), len(rev.compounds)

(236, 216)

In [28]:
get_As = lambda x : {k for k in x.keys() if k[0] == 'C'}
fwd_As = get_As(fwd.compounds)
rev_As = get_As(rev.compounds)
checkpoints = fwd_As & rev_As
len(checkpoints)

12

In [33]:
pk.reactions

{'R1af789f136d12c2bbf1e82054721cacfa9699e72e7bf984b74f7c40d5365bb44': {'_id': 'R1af789f136d12c2bbf1e82054721cacfa9699e72e7bf984b74f7c40d5365bb44',
  'Reactants': [(1, 'X281e38d790b809dc908f7c9749d5274a6d8280c4'),
   (1, 'Ce264bf1bbd482e436995855afdf2170ce7229f1d')],
  'Products': [(1, 'C99d9d917bb384cbe9fb3c54758d1c734c1514601'),
   (1, 'X151d45ce062c088336cff9e45bc138b6a63626fe')],
  'Operators': {'rule0003_083'},
  'SMILES_rxn': '(1) O=C(O)CCC(=O)C(=O)O + (1) NC(=O)C1=CN(C2OC(COP(=O)(O)OP(=O)(O)OCC3OC(n4cnc5c(N)ncnc54)C(OP(=O)(O)O)C3O)C(O)C2O)C=CC1 => (1) O=C(O)CCC(O)C(=O)O + (1) NC(=O)c1ccc[n+](C2OC(COP(=O)(O)OP(=O)(O)OCC3OC(n4cnc5c(N)ncnc54)C(OP(=O)(O)O)C3O)C(O)C2O)c1'},
 'Rd56a49666d721a40420b7efd71c6e474b223f4a34c6325f42b3750f406d77745': {'_id': 'Rd56a49666d721a40420b7efd71c6e474b223f4a34c6325f42b3750f406d77745',
  'Reactants': [(1, 'X281e38d790b809dc908f7c9749d5274a6d8280c4'),
   (1, 'Ce264bf1bbd482e436995855afdf2170ce7229f1d')],
  'Products': [(1, 'C2ec83be22e1a50ec52a8f183af0e